In [80]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import akshare as ak
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from database.downloader.downloader_base import DownloaderBase
import database.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [85]:
def plot_series_dist(series):
    data = series
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_label(self, stock_dataframe):
        N = 5 # 最大持仓周期 = N天，第N+1天开盘卖出
        df = stock_dataframe.copy()
        # 标签构建
        df['future_return'] = df['close'].shift(-N) / df['open'].shift(-1) - 1 # 计算第N日收益率
        # 极值处理
        df['future_return'] = np.clip(
            df['future_return'], 
            np.nanquantile(df['future_return'], 0.01), 
            np.nanquantile(df['future_return'], 0.99),
            )
        # 过滤第二天一字涨停情况
        df = df[df['high'].shift(-1) != df['low'].shift(-1)]
        return df[['datetime', 'future_return']]


    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_base = self.db_downloader._download_stock_history_info(stock_code, start_date, end_date) # 获取历史行情
        stock_factor_date = self.db_downloader._download_stock_factor_date_info() # 获取日期特征
        stock_factor_qlib = self.db_downloader._download_stock_factor_qlib_info(stock_code, start_date, end_date) # 获取量价特征
        stock_label = self._build_label(stock_base) # 构建label
        stock_df = stock_base.merge(stock_label, on=['datetime']).merge(stock_factor_date, on=['datetime']).merge(stock_factor_qlib, on=['stock_code', 'datetime']) # 整合数据
        stock_df = stock_df.dropna()
        return stock_df
    
    def _process_all_stock(self, code_type, start_date, end_date):
        # stock_code_list = list(ak.stock_info_a_code_name()['code'].unique()) # 获取A股所有股票列表
        # stock_code_list = list(ak.index_stock_cons("000905")['品种代码'].unique()) # 获取中证500的股票代码列表
        # stock_code_list = list(ak.index_stock_cons("000300")['品种代码'].unique()) # 获取沪深300的股票代码列表
        stock_code_list = list(ak.index_stock_cons(code_type)['品种代码'].unique()) # 获取中证50的股票代码列表
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {code_type} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)

In [86]:
db_conn = sqlite3.connect('../database/hh_quant.db')
db_downloader = DownloaderBase(db_conn, db_config)

proprocessor = PreProcessing(db_downloader=db_downloader)

## 使用Tensorflow

In [87]:
# 使用tensorflow处理原始数据
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.15.0


In [88]:
class Senet(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=3, seed=1024, **kwargs):
        super(Senet, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio
        self.seed = seed  

    def build(self, input_shape):
        self.field_size = len(input_shape)
        self.reduction_size = max(1, self.field_size // self.reduction_ratio)
        self.scale_layer = tf.keras.layers.Dense(units=self.reduction_size, activation='relu')
        self.expand_layer = tf.keras.layers.Dense(units=self.field_size, activation='relu')
        super(Senet, self).build(input_shape)

    def call(self, inputs, training=False):
        # print(f"Senet Is Training Mode: {training}")
        inputs = [tf.expand_dims(i, axis=1) for i in inputs]
        inputs = tf.concat(inputs, axis=1) # [B, N, dim]
        Z = tf.reduce_mean(inputs, axis=-1) # [B, N]
        A_1 = self.scale_layer(Z, training=training) # [B, X]
        A_2 = self.expand_layer(A_1, training=training) # [B, N]
        scale_inputs = tf.multiply(inputs, tf.expand_dims(A_2, axis=-1))
        output = scale_inputs + inputs # skip-connection
        return output # [B, N, dim]

    def get_config(self):
        config = super(Senet, self).get_config()
        config.update({
            'reduction_ratio': self.reduction_ratio,
            'seed': self.seed
        })
        return config

class Dnn(tf.keras.layers.Layer):
    def __init__(self, hidden_units=[64,32], activation="relu", dropout_rate=0.2, use_bn=True, seed=1024, **kwargs):
        super(Dnn, self).__init__(**kwargs)
        self.hidden_units = hidden_units
        self.activation = activation
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        self.seed = seed
        self.dense_layers = []
        self.dropout_layers = []
        self.bn_layers = []
        
    def build(self, input_shape):
        for units in self.hidden_units:
            self.dense_layers.append(tf.keras.layers.Dense(units=units, activation=self.activation))
            self.dropout_layers.append(tf.keras.layers.Dropout(rate=self.dropout_rate, seed=self.seed))
            if self.use_bn:
                self.bn_layers.append(tf.keras.layers.BatchNormalization())
        super(Dnn, self).build(input_shape)  # Be sure to call this at the end
    
    def call(self, inputs, training=False):
        # print(f"Dnn Is Training Mode: {training}")
        x = inputs
        for i in range(len(self.hidden_units)):
            x = self.dense_layers[i](x)
            if self.use_bn:
                x = self.bn_layers[i](x, training=training)
            x = self.dropout_layers[i](x, training=training)
        return x

    def get_config(self):
        config = super(Dnn, self).get_config()
        config.update({
            'hidden_units': self.hidden_units,
            'activation': self.activation,
            'dropout_rate': self.dropout_rate,
            'use_bn': self.use_bn,
            'seed': self.seed
        })
        return config
    
class QuantModel(tf.keras.Model):
	def __init__(self, config, **kwargs):
		super(QuantModel, self).__init__(**kwargs)
		self.config = config

		# 添加属性来存储预定义的层
		self.lookup_layers = {}
		self.embedding_layers = {}

        # 创建连续特征的离散化层和嵌入层
		for feature_name, boundaries in self.config.get("numeric_features_with_boundaries").items():
			self.lookup_layers[feature_name] = tf.keras.layers.Discretization(bin_boundaries=boundaries, output_mode='int', name=f'{feature_name}_lookup')
			self.embedding_layers[feature_name] = tf.keras.layers.Embedding(input_dim=len(boundaries) + 1, output_dim=self.config.get("feature_embedding_dims", 6), name=f'{feature_name}_embedding')
        # 创建整数特征的查找层和嵌入层
		for feature_name, vocab in self.config.get("integer_categorical_features_with_vocab").items():
			self.lookup_layers[feature_name] = tf.keras.layers.IntegerLookup(vocabulary=vocab, name=f'{feature_name}_lookup')
			self.embedding_layers[feature_name] = tf.keras.layers.Embedding(input_dim=len(vocab) + 1, output_dim=self.config.get("feature_embedding_dims", 6), name=f'{feature_name}_embedding')
		# 创建字符串特征的查找层和嵌入层
		for feature_name, vocab in self.config.get("string_categorical_features_with_vocab").items():
			self.lookup_layers[feature_name] = tf.keras.layers.StringLookup(vocabulary=vocab, name=f'{feature_name}_lookup')
			self.embedding_layers[feature_name] = tf.keras.layers.Embedding(input_dim=len(vocab) + 1, output_dim=self.config.get("feature_embedding_dims", 6), name=f'{feature_name}_embedding')

		# 任务Dnn层
		self.task_tower_list = []
		for task_type in self.config['task_type']:
			task_tower = tf.keras.Sequential([
				Senet(reduction_ratio=self.config.get('reduction_ratio', 3), seed=self.config.get('seed', 1024)),
				tf.keras.layers.Flatten(),
				Dnn(
					hidden_units=self.config.get('dnn_hidden_units', [64,32]), 
					activation=self.config.get('dnn_activation', 'relu'), 
					dropout_rate=self.config.get('dnn_dropout', 0.2), 
					use_bn=self.config.get('dnn_use_bn', True), 
					seed=self.config.get('seed', 1024),
				),
				tf.keras.layers.Dense(1, activation=None, name=task_type)
			])
			self.task_tower_list.append(task_tower)

	def call(self, inputs, training=False):
		# print(f"QuantModel Is Training Mode: {training}")
		# 确保inputs是一个字典类型，每个键值对应一个特征输入
		if not isinstance(inputs, dict): 
			raise ValueError('The inputs to the model should be a dictionary where keys are feature names.')
		encoded_features = []
    	# 现在使用已经实例化的层来编码输入
		for feature_name, feature_value in inputs.items():
        	# 使用预定义的查找层和嵌入层
			lookup_layer = self.lookup_layers[feature_name]
			embedding_layer = self.embedding_layers[feature_name]
			encoded_feature = embedding_layer(lookup_layer(feature_value))
			encoded_features.append(encoded_feature)
		
		# task任务塔
		logits_list = []
		for task_tower in self.task_tower_list:
			task_output = task_tower(encoded_features)
			logits_list.append(task_output)
		return logits_list
	
	def get_config(self):
		# 调用基类的get_config方法（如果基类实现了get_config）
		config = super(QuantModel, self).get_config()
        # 添加QuantModel特有的配置信息
		config.update({
            # 假设self.config是一个可序列化的字典，如果不是，你可能需要在这里适当地处理它
            'config': self.config
        })
		return config

In [94]:
def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date):
    # print(f"train_start: {train_start_date}, train_end: {train_end_date}, val_start: {val_start_date}, val_end: {val_end_date}")
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    return train_data, val_data

def transfer_data_type(df, columns, dtype):
    for col in columns:
        df[col] = df[col].astype(dtype)
    return df

def get_numeric_boundaries(series, num_bins=30):
    if series.nunique() < num_bins:
        boundaries = sorted(series.unique())
    else:
        boundaries = pd.qcut(series, num_bins, retbins=True, duplicates='drop')[1].tolist()
    return boundaries

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=True, batch_size=32):
    features = dataframe[feature_cols]
    labels = [dataframe[label_col] for label_col in label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), tuple(labels)))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(features))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

backtest_start_date = datetime.strptime('20200101', '%Y%m%d')
backtest_end_date = datetime.strptime('20240101', '%Y%m%d')
val_start_date = backtest_start_date
train_period = 6 # year
update_period = 6 # month
feature_config = {
    "target_feature_name": ["future_return"],
    "numeric_features": ['KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE0', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME0', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['weekday', 'day_of_month', 'month'],
    "string_categorical_features": ['day_of_week', 'season']
}
full_feature_names = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])

rolling_training_flag = True

batch_size = 128

epoch_date_list = []

while rolling_training_flag:
    val_end_date = val_start_date + relativedelta(months=update_period) - relativedelta(days=1)
    if val_start_date < backtest_end_date:
        train_start_date = val_start_date - relativedelta(years=train_period)
        train_end_date = val_start_date - relativedelta(days=1)
        current_date_period = [train_start_date, train_end_date, val_start_date, val_end_date]
        epoch_date_list.append([train_start_date, train_end_date, val_start_date, val_end_date])
        print(f"train_start: {train_start_date}, train_end: {train_end_date}, val_start: {val_start_date}, val_end: {val_end_date}")
         # 在这里处理滚动训练流程 ...
        df = proprocessor._process_all_stock(code_type='000016', start_date=train_start_date.strftime("%Y%m%d"), end_date=val_end_date.strftime("%Y%m%d"))
        train_data, val_data = extract_train_val_data(df, *current_date_period)

        train_ds = df_to_dataset(train_data, full_feature_names, feature_config.get('target_feature_name', []), shuffle=True, batch_size=batch_size)
        val_ds = df_to_dataset(val_data, full_feature_names, feature_config.get('target_feature_name', []), shuffle=False, batch_size=batch_size)

        model_config = {
             "seed": 1024,
             "reduction_ratio": 3,
            "dnn_hidden_units": [64,32],
            "dnn_activation": 'relu',
            "dnn_dropout": 0.1,
            "dnn_use_bn": True,
            "numeric_features_with_boundaries": {k: list(get_numeric_boundaries(train_data[k])) for k in feature_config.get('numeric_features', [])},
            "integer_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
            "string_categorical_features_with_vocab": {k: list(train_data[k].unique()) for k in feature_config.get('string_categorical_features', [])},
            "feature_embedding_dims": 6,
            "task_type": ['reg'],
        }
        model = QuantModel(model_config)

        monitor_indicator = 'val_loss'

        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor=monitor_indicator,
            verbose=2,
            patience=10,
            mode='min',
            restore_best_weights=True,
        )

        log_dir = "./logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

        # 配置optimizer
        # optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4) # for Mac M1/M2
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4) # for intel
        loss = [tf.keras.losses.MeanSquaredError()]
        model.compile(optimizer=optimizer, loss=loss)
        model.fit(
                train_ds, 
                validation_data=val_ds, 
                epochs=20,
                verbose=2,
                callbacks=[tensorboard_callback, early_stopping]
        )

        model_save_path = f'./models/saved_model/model_of_{val_start_date.strftime("%Y%m%d")}'
        model.save(model_save_path)
        # best_model = tf.keras.models.load_model('./best_model')

        model_red_result = model.predict(val_ds)
        output_df = val_data[['stock_code', 'stock_name', 'datetime']]
        output_df['future_return'] = val_data['future_return']
        output_df['future_return_pred'] = model_red_result[0]
        output_file_path = f'./predicts/stock_selection_results_{val_start_date.strftime("%Y%m%d")}.pkl'
        output_df.to_pickle(output_file_path)

        # 更新周期
        val_start_date += relativedelta(months=3) 
    else:
        rolling_training_flag=False # 结束滚动训练

train_start: 2014-01-01 00:00:00, train_end: 2019-12-31 00:00:00, val_start: 2020-01-01 00:00:00, val_end: 2020-06-30 00:00:00


Process: 000016 ...: 100%|██████████| 50/50 [00:10<00:00,  4.85it/s]


train_data_size: (59264, 199)
validation_data_size: (5000, 199)
Epoch 1/20
463/463 - 46s - loss: 0.6732 - val_loss: 0.1828 - 46s/epoch - 99ms/step
Epoch 2/20
463/463 - 15s - loss: 0.2790 - val_loss: 0.1053 - 15s/epoch - 32ms/step
Epoch 3/20
463/463 - 15s - loss: 0.1788 - val_loss: 0.0618 - 15s/epoch - 33ms/step
Epoch 4/20
463/463 - 14s - loss: 0.1251 - val_loss: 0.0371 - 14s/epoch - 31ms/step
Epoch 5/20
463/463 - 16s - loss: 0.0938 - val_loss: 0.0233 - 16s/epoch - 35ms/step
Epoch 6/20
463/463 - 16s - loss: 0.0705 - val_loss: 0.0151 - 16s/epoch - 34ms/step
Epoch 7/20
463/463 - 17s - loss: 0.0544 - val_loss: 0.0104 - 17s/epoch - 36ms/step
Epoch 8/20
463/463 - 16s - loss: 0.0426 - val_loss: 0.0074 - 16s/epoch - 34ms/step
Epoch 9/20
463/463 - 16s - loss: 0.0325 - val_loss: 0.0057 - 16s/epoch - 34ms/step
Epoch 10/20
463/463 - 15s - loss: 0.0255 - val_loss: 0.0046 - 15s/epoch - 32ms/step
Epoch 11/20
463/463 - 15s - loss: 0.0194 - val_loss: 0.0038 - 15s/epoch - 33ms/step
Epoch 12/20
463/463 -

INFO:tensorflow:Assets written to: ./models/saved_model/model_of_20200101/assets


40/40 [==============================] - 2s 17ms/step
train_start: 2014-04-01 00:00:00, train_end: 2020-03-31 00:00:00, val_start: 2020-04-01 00:00:00, val_end: 2020-09-30 00:00:00


...

INFO:tensorflow:Assets written to: ./models/saved_model/model_of_20200401/assets


43/43 [==============================] - 3s 16ms/step
train_start: 2014-07-01 00:00:00, train_end: 2020-06-30 00:00:00, val_start: 2020-07-01 00:00:00, val_end: 2020-12-31 00:00:00


Process: 000016 ...: 100%|██████████| 50/50 [00:09<00:00,  5.08it/s]


train_data_size: (59964, 199)
validation_data_size: (5567, 199)
Epoch 1/20
469/469 - 45s - loss: 0.7954 - val_loss: 0.1419 - 45s/epoch - 95ms/step
Epoch 2/20
469/469 - 16s - loss: 0.3044 - val_loss: 0.0778 - 16s/epoch - 34ms/step
Epoch 3/20
469/469 - 17s - loss: 0.1961 - val_loss: 0.0396 - 17s/epoch - 35ms/step
Epoch 4/20
469/469 - 16s - loss: 0.1475 - val_loss: 0.0219 - 16s/epoch - 35ms/step
Epoch 5/20
469/469 - 17s - loss: 0.1143 - val_loss: 0.0137 - 17s/epoch - 36ms/step
Epoch 6/20
469/469 - 16s - loss: 0.0963 - val_loss: 0.0094 - 16s/epoch - 34ms/step
Epoch 7/20
469/469 - 17s - loss: 0.0777 - val_loss: 0.0069 - 17s/epoch - 37ms/step
Epoch 8/20
469/469 - 15s - loss: 0.0633 - val_loss: 0.0055 - 15s/epoch - 31ms/step
Epoch 9/20
469/469 - 16s - loss: 0.0527 - val_loss: 0.0046 - 16s/epoch - 34ms/step
Epoch 10/20
469/469 - 16s - loss: 0.0433 - val_loss: 0.0040 - 16s/epoch - 34ms/step
Epoch 11/20
469/469 - 17s - loss: 0.0341 - val_loss: 0.0037 - 17s/epoch - 36ms/step
Epoch 12/20
469/469 -

KeyboardInterrupt: 

In [34]:
# %tensorboard --logdir ./logs/fit

(<tf.Tensor: shape=(), dtype=float64, numpy=-0.008333333333333304>,)
